In [ ]:
# After being trained, the model typically predicts 'no note' and hence its supplied ending is usually silence.
# This is likely due to 'no note' being the option with the highest frequency; meaning, when trying to minimize loss,
# insead of learning how to contunue/resolve a melody, the model simply chooses the single option which makes the loss
# decently low.
# I predict that with more training data (which is hard to come by as I have been creating it myself) the model might learn
# to compose music more interesting than silence.

In [1]:
from midiToArray import load_data, to_onehot # Custom library with functions for translating midi data to something ML can use
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dropout, Dense
import numpy as np

In [2]:
#Loading inputs and outputs from midi file

# If onehot is False, a measure of data will be an array wherein each index holds a midi note's value (eg. [60,62,64...])
# If onehot is True, each index of the measure-array will be a onehot vector (only supports C-Major notes from C4 to C5)
X, Y = load_data('data3.mid', eight_to_eight = False, memory_steps = 8, onehot = True, add_beat_vector=True, add_beat =False)

Y = Y.reshape((Y.shape[0], Y.shape[2]))

print('X shape:', X.shape, 'Y shape:', Y.shape)

X shape: (1632, 8, 18) Y shape: (1632, 10)


In [3]:
# Build model
model = keras.models.Sequential()


model.add(LSTM(32,  input_shape = X.shape[1:], return_sequences = True))

model.add(Dropout(0.2))

model.add(LSTM(32, return_sequences = False))

model.add(Dropout(0.2))

model.add(Dense(10, activation = 'softmax'))

opt = keras.optimizers.Adam(lr = 5e-4, decay = 5e-6)



model.compile(loss = 'categorical_crossentropy',
              optimizer = opt)

In [4]:
#train model
model.fit(X,Y, epochs= 5, batch_size = 32)

Epoch 1/5
51/51 [==============================] - 7s 8ms/step - loss: 2.2629
Epoch 2/5
51/51 [==============================] - 0s 8ms/step - loss: 1.9681
Epoch 3/5
51/51 [==============================] - 0s 8ms/step - loss: 1.8541
Epoch 4/5
51/51 [==============================] - 0s 8ms/step - loss: 1.7506
Epoch 5/5
51/51 [==============================] - 0s 8ms/step - loss: 1.7181


In [5]:
# Assumes consistent measure size
def predict(measures, length, add_beat_vector = False, add_beat = False):
    
    m_predict = to_onehot(measures, add_beat_vector, add_beat)
    
    m =  to_onehot(measures)
    
    shape = m.shape
    

    for j in range(length):
        #Get and append new note
        temp = model.predict(m_predict[:,j:]).argmax(axis=1)

        notes = np.zeros((shape[0],1, shape[2]))
        notes[np.arange(shape[0]), 0, temp] = 1
        m = np.concatenate((m,notes), axis=1)
        
        #Update input
        dim = shape[2]
        


        if add_beat_vector:
            dim += shape[1]
        if add_beat:
            dim += 1
            


        m2_predict = np.zeros((shape[0],m.shape[1], dim)) #'m.shape[1]' so it properly grows with the piece length
        
        m2_predict[:,:, :m.shape[2]] = m

        if add_beat_vector:
            for i in range(shape[0]):
                for j in range(m2_predict.shape[1]):
                    m2_predict[i][j][shape[2] + j%shape[1]] = 1


        if add_beat:
            for i in range(shape[0]):
                for j in range(shape[1]):
                    m2_predict[i][j][-1] = j
                


        
        m_predict = m2_predict

            
    return m

In [6]:
predict(np.array([[0,64,67,72,71,1,67,1]]), 8, add_beat_vector=True, add_beat= False)

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]])

In [7]:
predict(np.array([[0,64,67,72,71,1,67,1],
                 [60,1,0,62,64,1,0,62]]),
        8, add_beat_vector=True, add_beat= False)

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]],

       [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]

In [8]:
predict(np.array([[60,1,0,62,67,69,72,69]]), 8, add_beat_vector=True, add_beat= False)

array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]])

In [ ]:
# The predictions, as stated atop, are not very interesting. As only the latter eight of the onehot vectors are the
# predicted ones, it is clear that the last indice, the index representing 'no note' is frequently predicted